In [1]:
# importa las librerias necesarias para trabajar con pandas, numpy, BeautifulSoup, requests, selenium, y manejo de fechas
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup as bs 
import requests 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import datetime 
import time

In [2]:
# define la URL de la pagina web de los hoteles que se quiere scrapear
url_hoteles = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [3]:
# realiza una peticion HTTP GET a la URL de los hoteles y verifica el estado de la respuesta
res_hoteles = requests.get(url_hoteles) 
res_hoteles.status_code

200

In [4]:
# parsea el contenido HTML de la respuesta utilizando BeautifulSoup para facilitar la extraccion de informacion
sopa_hoteles = bs(res_hoteles.content, "html.parser")
sopa_hoteles

<!DOCTYPE html>

<html data-n-head-ssr="">
<head><script data-dtconfig="rid=RID_-1567294319|rpid=-1136404703|domain=accor.com|reportUrl=/rb_80be963f-a859-4808-9b2a-ceb8d44df738|app=1a152145cd696e21|featureHash=ICA7NVfhqrux|srsr=50000|xb=https:^bs/^bs/www^bs.google-analytics^bs.com^bs/collect^bs?.*^p .*/cdx/platform.html|rdnt=1|uxrgce=1|cuc=85aac0bj|mdl=mdcc5=20|mel=100000|expw=1|dpvc=1|md=mdcc2=cJSESSIONID,mdcc3=corg,mdcc4=bsessionStorage.userData,mdcc5=bCS_CONF.integrations_handler.replaylink,mdcc6=bCS_CONF.integrations_handler.sessionID,mdcc7=cplatform,mdcc8=fx-cdn-forward|lastModification=1740052810545|mdp=mdcc3|tp=500,50,0|agentUri=/ruxitagentjs_ICA7NVfhqrux_10285240307101407.js" src="/ruxitagentjs_ICA7NVfhqrux_10285240307101407.js" type="text/javascript"></script><script type="text/javascript">var kameleoonLoadingTimeout = 1000;window.kameleoonQueue = window.kameleoonQueue || [];window.kameleoonStartLoadTime = new Date().getTime();if (!document.getElementById('kameleoonLoadingStyl

In [5]:
# busca los elementos con la clase 'hotelblock__content' que contienen la informacion de los hoteles
hoteles = sopa_hoteles.find_all("div", {"class": "hotelblock__content"})
hoteles

[<div class="hotelblock__content" data-v-40d92af9=""><header class="title-block" data-v-40d92af9="" data-v-438ba70d=""><h2 class="title" data-v-438ba70d="" data-v-71eed73c=""><a class="title__link title__link" data-v-71eed73c="" href="/ssr/app/ibis/rates/8052/index.es.shtml?compositions=1&amp;dateIn=2025-02-21&amp;nights=1&amp;facets=open&amp;hideHotelDetails=false&amp;hideWDR=false&amp;destination=madrid-spain" target="" title="ibis Styles Madrid Prado">
     ibis Styles Madrid Prado
     <span class="title-block__stick-to-previous-text" data-v-438ba70d=""><span class="rating" data-v-399a1766="" data-v-438ba70d=""><span class="sr-only" data-v-399a1766="">3 Estrellas</span> <svg aria-hidden="true" class="icon rating__star" data-v-399a1766="" focusable="false" viewbox="0 0 7 7" xmlns="http://www.w3.org/2000/svg"><path d="M3.5 5.52 1.34 6.66l.41-2.41L0 2.55l2.42-.36L3.5 0l1.08 2.2L7 2.53l-1.75 1.7.41 2.42z"></path></svg><svg aria-hidden="true" class="icon rating__star" data-v-399a1766=""

In [6]:
# extrae el texto del enlace con la clase 'title__link title__link' de cada hotel
for hotel in hoteles:
    nombre = hotel.find("a", {"class": "title__link title__link"}).get_text()
nombre

'\n    ibis Styles Madrid City Las Ventas\n    4 Estrellas '

In [20]:
# formas por las que podria obtener el nombre del hotel por un lado y las estrellas pro el otro 
print(nombre.split("\n")[1].strip())
print(nombre.split("\n")[2].strip().split(" ")[0])

ibis Styles Madrid City Las Ventas
4


In [23]:
# intenta buscar el precio en el html 
for hotel in hoteles:
    precio = hotel.find("div", {"class": "hotelblock__content-priceblock"}).get_text()
precio

''

En este punto me doy cuenta de que de la clase title__link voy a poder obtener tanto el nombre del hotel como las estrellas y de que no se puede obtener el precio, pues no aparece en el html. Al igual que el nombre del hotel y las estrellas aparecen en el html, el precio no. Esto es porque está cargado dinámicamente con JavaScript y por tanto vamos a tener que obtenerlo con selenium en vez de con BeautifulSoup. Por lo tanto, voy a realizar todo el scraping con Selenium y comienzo sacando el precio 

In [30]:
# configuracion de Selenium para abrir el navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")  
driver = webdriver.Chrome(service=service, options=options)

# acceder a la pagina
driver.get(url_hoteles)

# esperar a que los elementos con los precios esten disponibles en la pagina
precios_hoteles = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "rate-details__price-wrapper"))
)

# iterar sobre cada precio encontrado y limpiarlo para obtener el valor numerico
for precio_hotel in precios_hoteles:
    precio_final = precio_hotel.text

# imprimir los precios obtenidos
print(precio_final)
print(precio_final.split('\n')[1].replace("€", ""))

# cerrar el navegador despues de obtener los datos
driver.quit()

Desde
77€
77


Ahora que se ha comprobado que con Selenium si que se saca el precio perfectamente, se genera una función que con Selenium extraiga todo: el nombre de los hoteles, las estrellas y los precios. Además, las estrellas no es lo que se comenzó recogiendo con BeautifulSoup, pues esas estrellas eran las estrellas del hotel, lo que se quiere obtener es las estrellas que resultan de la valoración / reseñas de todos los clientes. Por otro lado, incluyo en el diccionario una columna que es la fecha de reserva ya que esta columna va a ser la fecha en la que se realiza el scrapeo

In [33]:
def scrap_info_hoteles(url):
    # inicializa el diccionario para almacenar la informacion de los hoteles
    dictio_scrap = {
        "nombre_hotel": [],
        "estrellas": [],
        "precio_noche": [],
        "fecha_reserva": []
    }

    # configura el servicio de Selenium y las opciones para maximizar la ventana del navegador
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("--start-maximized")
    
    # inicializa el driver de Chrome para abrir la pagina web
    driver = webdriver.Chrome(service=service, options=options)

    # abre la URL proporcionada
    driver.get(url)

    try:
        # espera hasta que los precios de los hoteles esten visibles en la pagina
        precios_hoteles = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "rate-details__price-wrapper"))
        )

        # itera sobre los precios encontrados y se añaden al diccionario despues de limpiarlos y convertirlos a float
        for precio_hotel in precios_hoteles: 
            dictio_scrap["precio_noche"].append(float(precio_hotel.text.split('\n')[1].replace("€", "")))

        # espera hasta que los nombres de los hoteles esten visibles en la pagina
        nombre_hoteles = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "title"))
        )

        # itera sobre los nombres de los hoteles y se añaden al diccionario
        for nombre_hotel in nombre_hoteles: 
            dictio_scrap["nombre_hotel"].append(nombre_hotel.text.split("\n")[0])

        # espera hasta que las estrellas de los hoteles esten visibles en la pagina
        estrellas_hoteles = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "ratings__score"))
        )

        # itera sobre las estrellas de los hoteles y se añaden al diccionario. tambien se añade la fecha de la reserva con la fecha actual en formato datetime
        for estrella_hotel in estrellas_hoteles:
            dictio_scrap["estrellas"].append(float(estrella_hotel.text.split("/")[0]))
            dictio_scrap["fecha_reserva"].append(pd.Timestamp(datetime.now().date())) 

    except Exception as e:
        # en caso de error, se muestra un mensaje con la excepcion
        print("No se pudo encontrar los elementos:", e)

    # cierra el navegador al finalizar el scraping
    driver.quit()

    # devuelve el diccionario con los datos recopilados
    return dictio_scrap

In [ ]:
# llama a la funcion scrap_info_hoteles para obtener los datos de los hoteles desde la URL y luego los convierte en un DataFrame de pandas
dictio_final = scrap_info_hoteles(url_hoteles)
df_hoteles_competencia = pd.DataFrame(dictio_final)
df_hoteles_competencia

,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,ibis Styles Madrid Prado,4.7,161.0,2025-02-21
1,ibis budget Madrid Calle 30,4.4,110.0,2025-02-21
2,ibis Madrid Centro las Ventas,4.5,172.0,2025-02-21
3,ibis budget Madrid Centro las Ventas,4.3,119.0,2025-02-21
4,ibis budget Madrid Vallecas,4.3,102.0,2025-02-21
5,ibis Madrid Aeropuerto Barajas,4.4,116.0,2025-02-21
6,ibis Madrid Alcorcon Tresaguas,4.4,90.0,2025-02-21
7,ibis budget Madrid Aeropuerto,4.0,88.0,2025-02-21
8,ibis Madrid Alcobendas,4.4,85.0,2025-02-21
9,ibis budget Madrid Alcorcon Móstoles,4.5,77.0,2025-02-21


In [35]:
# comprueba que el tipo de dato de las columnas en el dataframe es el correcto 
df_hoteles_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_hotel   10 non-null     object        
 1   estrellas      10 non-null     float64       
 2   precio_noche   10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 452.0+ bytes


De esta forma ya se ha logrado obtener por Selenium los datos requeridos, pero se va a tratar de mejorar la función para que en vez de 3 bucles for, únicamente exista un bucle for. Es decir, que se parta de una clase que contenga toda la información de los hoteles, y a partir de ahí, se genere un bucle for para que vaya iterando por cada hotel y que de cada hotel recoja la información del nombre, valoración y precio de las mismas clases que ya hemos visto

In [3]:
# define la funcion scrap_info_hoteles que extrae la informacion de los hoteles desde una pagina web
def scrap_info_hoteles(url, sleep_time=5):
    
    # inicializa un diccionario vacio para almacenar la informacion de los hoteles
    dictio_scrap = {
        "nombre_hotel": [],  
        "estrellas": [],  
        "precio_noche": [], 
        "fecha_reserva": []  
    }
    
    # configura y abre el navegador con Selenium para hacer scraping de la pagina
    service = Service(ChromeDriverManager().install())  
    options = Options() 
    options.add_argument("--start-maximized")  
    driver = webdriver.Chrome(service=service, options=options)  

    driver.get(url)  # abre la URL proporcionada
    time.sleep(sleep_time)  # hace una pausa para asegurar que la pagina cargue completamente
    
    try:
        # busca todos los elementos de la pagina que corresponden a la informacion de cada hotel
        hoteles = driver.find_elements(By.CLASS_NAME, "hotelblock")

        for hotel in hoteles:
            # extrae y limpia el nombre del hotel
            nombre = hotel.find_element(By.CLASS_NAME, "title").text.split("\n")[0]    
            dictio_scrap["nombre_hotel"].append(nombre)  # añade el nombre al diccionario 

            # extrae y limpia el precio por hotel, convirtiendolo a tipo float
            precio_texto = hotel.find_element(By.CLASS_NAME, "rate-details__price-wrapper").text.split("\n")[1].replace("€", "")
            precio = float(precio_texto)
            dictio_scrap["precio_noche"].append(precio)  # añade el precio al diccionario

            # extrae y limpia las estrellas, convirtiendolo a tipo float
            estrellas_texto = hotel.find_element(By.CLASS_NAME, "ratings__score").text.split("/")[0]
            estrellas = float(estrellas_texto)
            dictio_scrap["estrellas"].append(estrellas)  # añade las estrellas al diccionario

            # añade la fecha de reserva al diccionario como la fecha actual
            dictio_scrap["fecha_reserva"].append(pd.Timestamp(datetime.now().date()))

    except Exception as e:
        # en caso de error, muestra el mensaje de error
        print("Error al extraer la información:", e)

    driver.quit()  # cierra el navegador
    
    # retorna el diccionario con toda la informacion extraida
    return dictio_scrap


In [4]:
# llama a la funcion scrap_info_hoteles para obtener los datos de los hoteles desde la URL y luego los convierte en un DataFrame de pandas
dictio_final = scrap_info_hoteles(url_hoteles, sleep_time=5)
df_hoteles_competencia = pd.DataFrame(dictio_final)
df_hoteles_competencia

,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,ibis Styles Madrid Prado,4.7,161.0,2025-02-21
1,ibis budget Madrid Calle 30,4.4,110.0,2025-02-21
2,ibis Madrid Centro las Ventas,4.5,172.0,2025-02-21
3,ibis budget Madrid Centro las Ventas,4.3,119.0,2025-02-21
4,ibis budget Madrid Vallecas,4.3,102.0,2025-02-21
5,ibis Madrid Aeropuerto Barajas,4.4,116.0,2025-02-21
6,ibis Madrid Alcorcon Tresaguas,4.4,90.0,2025-02-21
7,ibis budget Madrid Aeropuerto,4.0,88.0,2025-02-21
8,ibis Madrid Alcobendas,4.4,85.0,2025-02-21
9,ibis budget Madrid Alcorcon Móstoles,4.5,77.0,2025-02-21


In [5]:
# comprueba que el tipo de dato de las columnas en el dataframe es el correcto 
df_hoteles_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre_hotel   10 non-null     object        
 1   estrellas      10 non-null     float64       
 2   precio_noche   10 non-null     float64       
 3   fecha_reserva  10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 452.0+ bytes


Observamos como el dataframe obtenido al final es el mismo, pero de una manera más eficiente

In [6]:
# guarda el dataframe obtenido del web scraping en un archivo pickle 
df_hoteles_competencia.to_pickle("../data/datos_extraidos/nombre_estrellas_precio.pickle")